In [32]:
!pip install -q pypdf

In [2]:
!pip install -q python-dotenv

In [33]:
!pip install -q transformers
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir # remove args for working in cpu
!pip install -q llama-index
!pip install -q llama-index
!pip -q install sentence-transformers

In [34]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


In [24]:
documents = SimpleDirectoryReader("/content/documents").load_data()

In [25]:
enhanced_documents = []
for doc in documents:
  file_name = doc.metadata.get('file_name', '')
  enhanced_documents.append(doc)

In [40]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf?download=true',
    temperature=0.01,
    max_new_tokens=1500,
    context_window=4000,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True
)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [8]:
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index import ServiceContext
# embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-base")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [26]:
service_context = ServiceContext.from_defaults(
    chunk_size=1000,
    llm=llm,
    embed_model=embed_model,
    chunk_overlap=20
)

In [27]:
index = VectorStoreIndex.from_documents(enhanced_documents, service_context=service_context)

In [28]:
query_engine = index.as_query_engine(index_id="resume")

In [29]:
# job_description_file = SimpleDirectoryReader("/content/jd").load_data()
# jd=[]
# for jd_doc in job_description_file:
#   file_name = jd_doc.metadata.get('file_name', '')
#   # if file_name in metadata_map:
#   #     doc.metadata.update(metadata_map[file_name])
#   jd.append(jd_doc)
jd ="Minimum of 2 years of professional experience as a Machine Learning Engineer\
Strong proficiency in programming languages such as Python, and experience with relevant libraries (e.g., TensorFlow, PyTorch, scikit-learn).\
Solid understanding of machine learning concepts, algorithms, and statistical modeling.\
Experience with deploying machine learning models in production environments.\
Proficient in data preprocessing, feature engineering, and model evaluation techniques.\
should have experience in docker.\
Excellent problem-solving skills and the ability to work on diverse and complex problems."

In [30]:
query_result = query_engine.query(f"Give the name of top 3 resumes that matches with the job description: {jd} and provide score for the matched document")
response_text = str(query_result)
print(response_text)

Llama.generate: prefix-match hit


 Based on the given job description and the context information provided, I have identified three resumes that appear to match the requirements:

1. Kavinsharaj V (file_path: /content/documents/kavin.pdf)
   - He has a minimum of 2 years of professional experience as a Machine Learning Engineer.
   - He is proficient in programming languages such as Python and has experience with relevant libraries like scikit-learn.
   - He has a solid understanding of machine learning concepts, algorithms, and statistical modeling.
   - He mentions experience with deploying machine learning models in production environments using AWS Lambda.
   - He is proficient in data preprocessing and feature engineering techniques.
   - He also mentions experience with containerization (Docker).

2. Aamruth Venkatesh (file_path: /content/documents/Aamruth_Venkatesh_resume.pdf)
   - He has a minimum of 2 years of professional experience as a Data Scientist, which includes implementing cutting-edge Machine Learnin

AttributeError: ignored